Analisis items publicados con oferta MeLi

In [74]:
#Importo las librerias que me interesan
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import  mean_absolute_error, mean_squared_error


BASE_URI = 'https://api.mercadolibre.com'



ModuleNotFoundError: No module named 'statsmodels'

In [2]:
def get_site_categories(site):
    '''Funcion para traer la info de las categorias que hay en un site'''
    urls = f'{BASE_URI}/sites/{site}/categories#json'
    response = requests.get(urls)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception('Error making get api call categories from {}.'.format(site))

def get_subcategories(category):
    '''Funcion para traer la info de las subcategoriascategorias que hay en un site'''
    urls = f'{BASE_URI}/categories/{category}'
    response = requests.get(urls)

    if response.status_code == 200:
        return response.json()['children_categories']
    else:
        raise Exception('Error making get api call subcategories from {}.'.format(category))

def get_items(site,subcategory):
    '''Funcion para traer los items que hay en un site'''
    urls = f'{BASE_URI}/sites/{site}/search?category={subcategory}'
    response = requests.get(urls)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception('Error making get api call items from {}.'.format(site))
        
def get_item_info(itemid):
    '''Funcion para traer la info de los items'''
    urls = f'{BASE_URI}/items/{itemid}'
    response = requests.get(urls)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception('Error making get api call item info for item_id {}.'.format(itemid))

def get_categorie_from_name(catjson, elementname):
    '''Funcion para buscar el nombre de una categoria'''
    for subcat in catjson:
        if subcat['name']!= elementname:
            continue
        else:
            return print(subcat['id'])
    raise Exception('Id not found')
    
def check_na(data):
    # return the columns having NAs, sorted descendingly by their number of NAs
    na_count = [sum(data[ff].isnull()) for ff in data.columns]
    return pd.DataFrame({'column': data.columns, 'na_count': na_count}).\
              query('na_count > 0').sort_values('na_count', ascending=False)  

In [3]:
#Creo un dataframe para despues agregar los datos
data = pd.DataFrame()

selectedcolumns = ['id',
                   'site_id',
                   'price',
                   'sale_price',
                   'currency_id',
                   'available_quantity',
                   'sold_quantity',
                   'buying_mode',
                   'listing_type_id',
                   'condition',
                   'accepts_mercadopago',
                   'original_price',
                   'category_id',
                   'domain_id',
                   'seller.eshop.eshop_status_id',
                   'seller.seller_reputation.power_seller_status',
                   'seller.seller_reputation.level_id',
                   'seller.seller_reputation.metrics.cancellations.rate',
                   'seller.seller_reputation.metrics.claims.rate',
                   'seller.seller_reputation.metrics.delayed_handling_time.rate',
                   'seller.seller_reputation.transactions.canceled',
                   'seller.seller_reputation.transactions.period',
                   'seller.seller_reputation.transactions.ratings.negative',
                   'seller.seller_reputation.transactions.ratings.neutral',
                   'seller.seller_reputation.transactions.ratings.positive',
                   'seller.seller_reputation.transactions.completed',
                   'installments.quantity', 
                   'installments.rate',
                   'address.state_id',
                   'shipping.free_shipping',
                   'shipping.mode',
                   'shipping.store_pick_up',
                   'categ_id_l1',
                   'categ_name_l1',
                   'categ_id_l2',
                   'categ_name_l2'
                  ]

In [4]:
#defino los sites que voy a analizar

sites=['MLA']

#Busco las categorias del site, subcategorias y los items que se encuentran publicados y los agrego al DF
for site in sites:
    categories = get_site_categories(site)

    for categ in categories:
        subcategories = get_subcategories(categ['id'])

        for subcat in subcategories:
            subcatitems = get_items(site,subcat['id']) 

            dftemp = pd.json_normalize(
                subcatitems,
                record_path = ['results'],
                errors = 'ignore')
            dftemp['categ_id_l1']=str(categ['id'])
            dftemp['categ_name_l1']=str(categ['name'])
            dftemp['categ_id_l2']=str(subcat['id'])
            dftemp['categ_name_l2']=str(subcat['name'])
            data = data.append(dftemp)
            data = data[selectedcolumns]





In [5]:
#Elegir columnas relevantes
#Conseguir el DF para el modelo
#Dummies
#separar el DF en train y test
#regresion lineal?
#lasso?
#random forest
#tunear el modelo
#testear

data.head()

,id,site_id,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,...,installments.quantity,installments.rate,address.state_id,shipping.free_shipping,shipping.mode,shipping.store_pick_up,categ_id_l1,categ_name_l1,categ_id_l2,categ_name_l2
0,MLA6197997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos
1,MLA10292229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos
2,MLA15070272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos
3,MLA15089343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos
4,MLA11386902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos


In [56]:
features_num = ['price', #num
           'available_quantity', #num
           'sold_quantity', #num
           'seller.eshop.eshop_status_id', #num
           'seller.seller_reputation.metrics.cancellations.rate', #num
           'seller.seller_reputation.metrics.claims.rate', #num
           'seller.seller_reputation.metrics.delayed_handling_time.rate', #num
           'seller.seller_reputation.transactions.ratings.negative', #num
           'seller.seller_reputation.transactions.ratings.neutral', #num
           'seller.seller_reputation.transactions.ratings.positive', #num
           'seller.seller_reputation.transactions.completed' #num
          ]
features_str = ['listing_type_id', #str
#           'domain_id', #str
           'seller.seller_reputation.power_seller_status', #str
           'seller.seller_reputation.level_id', #str
           'shipping.mode', #str
           'categ_id_l2', #str
          ]
features_bool = ['shipping.free_shipping', #bool
           'shipping.store_pick_up', #bool
           'offer', #bool
          ]

features_total = ['price', #num
           'available_quantity', #num
           'sold_quantity', #num
           'seller.eshop.eshop_status_id', #num
           'seller.seller_reputation.metrics.cancellations.rate', #num
           'seller.seller_reputation.metrics.claims.rate', #num
           'seller.seller_reputation.metrics.delayed_handling_time.rate', #num
           'seller.seller_reputation.transactions.ratings.negative', #num
           'seller.seller_reputation.transactions.ratings.neutral', #num
           'seller.seller_reputation.transactions.ratings.positive', #num
           'seller.seller_reputation.transactions.completed', #num
           'listing_type_id', #str
           'domain_id', #str
           'seller.seller_reputation.power_seller_status', #str
           'seller.seller_reputation.level_id', #str
           'shipping.mode', #str
           'categ_id_l2', #str
           'shipping.free_shipping', #bool
           'shipping.store_pick_up', #bool
           'offer' #bool
]
#Acomodo la info que me interesa
data_mod = data
data_mod.dropna()
data_mod['offer'] = np.where(data_mod['original_price']>0,True,False)

for col in features_num:
    data_mod[col] = pd.to_numeric(data_mod[col])
    


In [17]:
data_mod['listing_type_id'].unique()

array([nan, 'gold_special', 'gold_pro', 'gold_premium', 'gold', 'free',
       'silver', 'bronze'], dtype=object)

In [18]:
#Si hago onehot encoidng con tantas categorias se va a hacer muy grande el data set
data_mod['category_id'].describe()

count         16519
unique         1968
top       MLA109027
freq             70
Name: category_id, dtype: object

In [19]:
#modelo simplificado con esta categoría
data_mod['categ_id_l2'].describe()

count         20207
unique          410
top       MLA432888
freq             50
Name: categ_id_l2, dtype: object

In [20]:
#modelo simplificado con esta categoría
data_mod['address.state_id'].describe()

count     16519
unique       48
top        AR-C
freq       7974
Name: address.state_id, dtype: object

In [22]:
target = 'sold_quantity'

na_checker = check_na(data_mod[features_num]).sort_values('na_count', ascending=False).\
    reset_index(drop=True)
print(na_checker)

Empty DataFrame
Columns: [column, na_count]
Index: []


In [23]:
data_mod['seller.eshop.eshop_status_id'].fillna(0)

0     1.19944
1     1.19944
2     1.19944
3     1.19944
4     1.19944
       ...   
45    1.19944
46    1.19944
47    1.19944
48    1.19944
49    1.00000
Name: seller.eshop.eshop_status_id, Length: 20207, dtype: float64

In [24]:
data_mod.head()

,id,site_id,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,condition,...,installments.rate,address.state_id,shipping.free_shipping,shipping.mode,shipping.store_pick_up,categ_id_l1,categ_name_l1,categ_id_l2,categ_name_l2,offer
0,MLA6197997,NaN,123888.352049,NaN,NaN,940.984261,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos,False
1,MLA10292229,NaN,123888.352049,NaN,NaN,940.984261,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos,False
2,MLA15070272,NaN,123888.352049,NaN,NaN,940.984261,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos,False
3,MLA15089343,NaN,123888.352049,NaN,NaN,940.984261,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos,False
4,MLA11386902,NaN,123888.352049,NaN,NaN,940.984261,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,MLA5725,Accesorios para Vehículos,MLA4711,Acc. para Motos y Cuatriciclos,False


In [26]:
data_mod[features_num] = data_mod[features_num].fillna(data_mod[features_num].mean())

In [27]:
na_checker = check_na(data_mod[features_num]).sort_values('na_count', ascending=False).\
    reset_index(drop=True)
print(na_checker)

Empty DataFrame
Columns: [column, na_count]
Index: []


In [28]:
data_final = data_mod[data_mod['sold_quantity'].notna()]

In [31]:
data_final = data_final[features_total]

In [42]:
data_final.head()

,price,available_quantity,sold_quantity,seller.eshop.eshop_status_id,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.completed,listing_type_id,domain_id,seller.seller_reputation.power_seller_status,seller.seller_reputation.level_id,shipping.mode,categ_id_l2,shipping.free_shipping,shipping.store_pick_up,offer
6,6599.00,1385.0,2323.0,1.00000,0.0058,0.0054,0.0200,0.02,0.01,0.97,19591.0,gold_special,MLA-MOTORCYCLE_GLOVES,platinum,5_green,me2,MLA4711,True,False,False
7,5561.00,9993.0,3357.0,1.19944,0.0038,0.0163,0.0547,0.09,0.05,0.86,30821.0,gold_special,MLA-MOTORCYCLE_SUITS,platinum,5_green,me2,MLA4711,True,False,True
8,3195.06,98.0,2103.0,1.19944,0.0043,0.0059,0.0059,0.02,0.02,0.96,29453.0,gold_special,MLA-MOTORCYCLE_INTERCOMMUNICATORS,platinum,5_green,me2,MLA4711,False,False,True
9,1325.00,303.0,1197.0,1.19944,0.0038,0.0163,0.0547,0.09,0.05,0.86,30821.0,gold_special,MLA-MOTORCYCLE_GLOVES,platinum,5_green,me2,MLA4711,False,False,True
10,6545.00,4.0,1469.0,1.00000,0.0000,0.0054,0.0041,0.00,0.01,0.99,7555.0,gold_special,MLA-MOTORCYCLE_SADDLEBAGS,platinum,5_green,me2,MLA4711,True,False,False


In [39]:
data_dummies = pd.get_dummies(data_final[features_str])

In [58]:
data_encodear = data_final[features_str]

In [59]:
data_dummies = pd.get_dummies(data_encodear)

In [60]:
data_dummies.head()

,listing_type_id_bronze,listing_type_id_free,listing_type_id_gold,listing_type_id_gold_premium,listing_type_id_gold_pro,listing_type_id_gold_special,listing_type_id_silver,seller.seller_reputation.power_seller_status_gold,seller.seller_reputation.power_seller_status_platinum,seller.seller_reputation.power_seller_status_silver,...,categ_id_l2_MLA8959,categ_id_l2_MLA9007,categ_id_l2_MLA9038,categ_id_l2_MLA91058,categ_id_l2_MLA91083,categ_id_l2_MLA91088,categ_id_l2_MLA93412,categ_id_l2_MLA93430,categ_id_l2_MLA95955,categ_id_l2_MLA9808
6,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
data_final = data_final[features_num+features_bool]

In [63]:
data_final.head()

,price,available_quantity,sold_quantity,seller.eshop.eshop_status_id,seller.seller_reputation.metrics.cancellations.rate,seller.seller_reputation.metrics.claims.rate,seller.seller_reputation.metrics.delayed_handling_time.rate,seller.seller_reputation.transactions.ratings.negative,seller.seller_reputation.transactions.ratings.neutral,seller.seller_reputation.transactions.ratings.positive,seller.seller_reputation.transactions.completed,shipping.free_shipping,shipping.store_pick_up,offer
6,6599.00,1385.0,2323.0,1.00000,0.0058,0.0054,0.0200,0.02,0.01,0.97,19591.0,True,False,False
7,5561.00,9993.0,3357.0,1.19944,0.0038,0.0163,0.0547,0.09,0.05,0.86,30821.0,True,False,True
8,3195.06,98.0,2103.0,1.19944,0.0043,0.0059,0.0059,0.02,0.02,0.96,29453.0,False,False,True
9,1325.00,303.0,1197.0,1.19944,0.0038,0.0163,0.0547,0.09,0.05,0.86,30821.0,False,False,True
10,6545.00,4.0,1469.0,1.00000,0.0000,0.0054,0.0041,0.00,0.01,0.99,7555.0,True,False,False


In [64]:
data_final = data_final.join(data_dummies)

In [66]:
X = data_final.drop('sold_quantity', axis = 1)
y = data_final['sold_quantity'].values

4.0

In [73]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [77]:
#Regresion lineal
linear_reg = LinearRegression()
linear_reg.fit(X,y).summary()

MemoryError: Unable to allocate 17.7 GiB for an array with shape (432, 5509491) and data type object